In [18]:
import numpy as np
import plotly.express as px
from keras import Input, Model
from keras.src.layers import Dense
from keras.src.optimizers import Adam
import plotly.graph_objects as go

In [36]:
series = np.sin(0.1 * np.arange(200)) - np.random.randn(200) * 0.1

px.line(series)

In [37]:
T = 10
X = []
Y = []

for t in range(len(series) - T):
    x = series[t: t + T]
    y = series[t + T]
    X.append(x)
    Y.append(y)

X = np.array(X).reshape(-1, T)
Y = np.array(Y)
print(f"X.shape: {X.shape}\nY.shape: {Y.shape}")
N = len(X)

X.shape: (190, 10)
Y.shape: (190,)


In [38]:
i = Input(shape=(T,))
d = Dense(units=1)(i)
model = Model(i, d)
model.compile(loss="mse", optimizer=Adam(learning_rate=0.01))

history = model.fit(x=X[:-N // 2], y=Y[:-N // 2], epochs=150, validation_data=(X[-N // 2:], Y[-N // 2:]))

Epoch 1/150
3/3 [==============================] - 1s 201ms/step - loss: 0.0678 - val_loss: 0.0536
Epoch 2/150
3/3 [==============================] - 0s 102ms/step - loss: 0.0557 - val_loss: 0.0459
Epoch 3/150
3/3 [==============================] - 0s 76ms/step - loss: 0.0473 - val_loss: 0.0390
Epoch 4/150
3/3 [==============================] - 0s 71ms/step - loss: 0.0418 - val_loss: 0.0339
Epoch 5/150
3/3 [==============================] - 0s 58ms/step - loss: 0.0379 - val_loss: 0.0309
Epoch 6/150
3/3 [==============================] - 0s 133ms/step - loss: 0.0354 - val_loss: 0.0295
Epoch 7/150
3/3 [==============================] - 0s 59ms/step - loss: 0.0343 - val_loss: 0.0291
Epoch 8/150
3/3 [==============================] - 0s 45ms/step - loss: 0.0338 - val_loss: 0.0292
Epoch 9/150
3/3 [==============================] - 0s 46ms/step - loss: 0.0334 - val_loss: 0.0292
Epoch 10/150
3/3 [==============================] - 0s 48ms/step - loss: 0.0333 - val_loss: 0.0291
Epoch 11/150
3/3

In [39]:
def draw_loss(history):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history['loss'], name="loss"))
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history['val_loss'], name="val_loss"))
    fig.show()


draw_loss(history)

In [23]:
# # Wrong predictions
# validation_targets = Y[-N // 2:]
# validation_predictions = []
# 
# i = -N // 2
# 
# while len(validation_predictions) < len(validation_targets):
#     p = model.predict(X[i].reshape(1, -1), verbose=0)[0, 0]
#     i += 1
# 
#     validation_predictions.append(p)

In [71]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(y=validation_targets, name="targets"))
# fig.add_trace(go.Scatter(y=validation_predictions, name="predictions"))
# fig.show()

In [44]:
# Correct predictions
validation_targets = Y[-N // 2:]
validation_predictions = []

last_x = X[-N // 2]

while len(validation_predictions) < 250:
    p = model.predict(last_x.reshape(1, -1), verbose=0)[0, 0]

    validation_predictions.append(p)

    last_x = np.roll(last_x, -1)
    last_x[-1] = p

In [45]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=validation_targets, name="targets"))
fig.add_trace(go.Scatter(y=validation_predictions, name="predictions"))
fig.show()